In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sys
from tensorflow.keras import layers, models # type: ignore
from sklearn.model_selection import train_test_split
import pickle

In [3]:
# Function to train the CNN model (or load a pre-trained one)
def build_and_train_model():
    (x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.mnist.load_data() #Load Data
    x_train_full = x_train_full.astype('float32') / 255.0 #Normalize //from 0 to 1
    x_test = x_test.astype('float32') / 255.0 #Normalize
    x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=42)

    x_train = x_train.reshape(-1, 28, 28, 1)
    x_val = x_val.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)

    # Build the CNN model
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.6), #to prevent overfitting
        layers.Dense(10, activation='softmax')  # 10 classes for digits 0-9
    ])

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

    # Save the model
    model.save('attendance_digit_model.h5')

In [6]:
from tensorflow.keras.models import load_model 
build_and_train_model()
model = load_model('attendance_digit_model.h5')

c:\Users\Computer Store\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 85s 56ms/step - accuracy: 0.8502 - loss: 0.5289 - val_accuracy: 0.9840 - val_loss: 0.0548
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 73s 48ms/step - accuracy: 0.9767 - loss: 0.0827 - val_accuracy: 0.9881 - val_loss: 0.0405
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 68s 45ms/step - accuracy: 0.9845 - loss: 0.0530 - val_accuracy: 0.9880 - val_loss: 0.0404
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 68s 46ms/step - accuracy: 0.9871 - loss: 0.0416 - val_accuracy: 0.9878 - val_loss: 0.0439
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 69s 46ms/step - accuracy: 0.9895 - loss: 0.0338 - val_accuracy: 0.9902 - val_loss: 0.0388
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 68s 46ms/step - accuracy: 0.9910 - loss: 0.0280 - val_accuracy: 0.9918 - val_loss: 0.0341
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 68s 46ms/step - accuracy: 0.9929 - loss: 0.0228 - val_accuracy: 0.9906 - val_loss: 0.0413
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 68s 45ms/step - accuracy: 0.9943 -

In [7]:
from PIL import Image

def load_and_preprocess_image(path):
    img = Image.open(path).convert('L')           # Convert to grayscale
    img = img.resize((28, 28))                    # Resize to 28x28
    img_array = np.array(img) / 255.0             # Normalize
    img_array = 1 - img_array                     # Invert colors if white digit on black bg
    img_array = img_array.reshape(1, 28, 28, 1)   # Add batch and channel dims
    return img_array

In [10]:
image = load_and_preprocess_image('test44.png')
prediction = model.predict(image)
predicted_digit = np.argmax(prediction)
print("Predicted Digit:", predicted_digit)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted Digit: 1
